# Scrape repo stats

In [ ]:
%reset

In [5]:
import os

In [9]:
# load packages 
import os
import psycopg2 as pg
from sqlalchemy import create_engine
import pandas as pd
import requests as r
import string 
import json
import base64
import urllib.request
import itertools 
import numpy as np
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import warnings
from github import Github, RateLimitExceededException, BadCredentialsException, BadAttributeException, GithubException, UnknownObjectException, BadUserAgentException

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
#test clean repo name
#slug name array
slug = np.array(["repo", "repo  ", " repo", "  repo  ", "  re po  "])
print(slug)
#star count array
star = np.array([1,2,3,4,5])
print(star)

#combine two arrays to a dataframe
df = pd.DataFrame(dict(slug = slug, star = star))
print(df)

#extract slug names to a list 
raw_slugs = df["slug"].tolist()
print(raw_slugs)

#method 1 to exclude leading and ending space
raw_slugs_m1 = raw_slugs
raw_slugs_m1 = [s.strip() for s in raw_slugs_m1]
print(raw_slugs_m1)

#method 2 to exclude leading and ending space
slugs = []
for s in raw_slugs:
    slugs.append(s.strip())

print(slugs)

['repo' 'repo  ' ' repo' '  repo  ' '  re po  ']
[1 2 3 4 5]
        slug  star
0       repo     1
1     repo       2
2       repo     3
3     repo       4
4    re po       5
['repo', 'repo  ', ' repo', '  repo  ', '  re po  ']
['repo', 'repo', 'repo', 'repo', 're po']
['repo', 'repo', 'repo', 'repo', 're po']


In [3]:
#def string_is_empty(myString):
#    if myString and myString.strip():
#        return False
#    return True

def string_is_empty(myString):
    if myString == None:
         return True
    else:
        return False

print(string_is_empty('  '))
print(string_is_empty('')) 
print(string_is_empty(None))
print(string_is_empty('some text'))

False
False
True
False


In [29]:
def stars_count(url):
    html = r.get(url).text
    #soup = BeautifulSoup(html, 'lxml') 
    soup = BeautifulSoup(html, 'html.parser')
    stars_class = "social-count js-social-count"
    stars_text = soup.find('a', class_ = stars_class)
    if string_is_empty(stars_text) == True:
        raise UnknownObjectError("unknown object error")
    else:
        return stars_text.text.strip()


url1 = "https://github.com/tidyverse/ggplot2"
print(stars_count(url1))

url2 = "https://www.geeksforgeeks.org/python-program-to-check-if-string-is-empty-or-not/"
print(stars_count(url2))

#na but page exists
#url = "https://github.com/egeor90/stocks/"

#actual not found pages
#url ="https://github.com/codez0mb1e/HomeCreditDefaultRisk/"
print(stars_count(url))

4.9k


NameError: name 'UnknownObjectError' is not defined

In [294]:

#example
#url = "https://www.geeksforgeeks.org/python-program-to-check-if-string-is-empty-or-not/"
#url = "https://github.com/tidyverse/ggplot2"


#failed to recognized page
#url = "https://github.com/dempseynoel/nrstations/"

#actual not found page
url = "https://github.com/codez0mb1e/HomeCreditDefaultRisk/"


html = r.get(url).text

soup = BeautifulSoup(html, 'html.parser')
#print(soup)
stars_class = "social-count js-social-count"
stars_text = soup.find('a', class_ = stars_class)



#stars_text == None
#print(stars_text)
#string_is_empty(stars_text) 

#stars_text.text.strip()

In [295]:
myPath = os.getcwd()

print(myPath + "\n")

/sfs/qumulo/qhome/zz3hs/git/dspg21oss/src/01_scrape_readmes



In [15]:
# connect to the database, download data, limit to top 5000 repos that have the highest number of commits
connection = pg.connect(host = 'postgis1', database = 'sdad', 
                        user = os.environ.get('db_user'), 
                        password = os.environ.get('db_pwd'))

raw_slug_data = '''SELECT * FROM gh_2007_2020.repos_ranked LIMIT 10'''

# convert to a dataframe, show how many missing we have (none)
raw_slug_data = pd.read_sql_query(raw_slug_data, con=connection)
print(raw_slug_data.head())
print(raw_slug_data.shape)
print(raw_slug_data.isna().sum())

raw_slugs= raw_slug_data["slug"].tolist()
#slugs = [raw_slugs.strip(' ') for raw_slugs in slugs]
slugs = []
for s in raw_slugs:
    slugs.append(s.strip())
    
#print(len(slugs))
#print(slugs)


                                 id          spdx  \
0  MDEwOlJlcG9zaXRvcnkyNzUwNzAwMzc=  BSD-3-Clause   
1  MDEwOlJlcG9zaXRvcnkxOTM0MzEyNTI=  BSD-3-Clause   
2  MDEwOlJlcG9zaXRvcnkyODUxOTgyOTQ=       GPL-2.0   
3  MDEwOlJlcG9zaXRvcnkzMDU2MTkzMjA=       GPL-2.0   
4  MDEwOlJlcG9zaXRvcnk4Mjk0MDUzOA==           MIT   

                                      slug           createdat  \
0                  dbuskariol-org/chromium 2020-06-26 04:04:29   
1                                 cjbd/src 2019-06-24 04:03:03   
2  firemax13/android_kernel_sm6150_unified 2020-08-05 06:17:00   
3                      firemax13/a80kernel 2020-10-20 07:04:11   
4  eugene-matvejev/ultimate-commit-machine 2017-02-23 15:24:09   

                                         description primarylanguage  \
0  The official GitHub mirror of the Chromium source            None   
1                                                src            None   
2  Samsung Galaxy A71 & A80 Unified Kernel Source...               C

In [16]:
star = []
for slug in slugs:
    url =f'https://github.com/{slug}/'
    star.append(stars_count(url))
    
print(star)


['0', None, None, None, '1', '119', '0', '2', '0', '1']


In [19]:

def pull_repo_stats():
    df_repo_stats = pd.DataFrame()
    for slug in slugs:
        while True:
            try:
                #if false, retry until true, max number of retry is 20 times
                #g = Github(access_token, retry = 20, timeout = 15)
                url =f'https://github.com/{slug}/'
                df_repo_stats = df_repo_stats.append({
                    "slug": slug,
                    'stars': stars_count(url)
                }, ignore_index = True)
            except RateLimitExceededException as e:
                print(e.status)
                print('Rate limit exceeded --', slug)
                time.sleep(300)
                continue
            except UnknownObjectError as e:
                print(e.status)
                print('Unknown object exception --', slug)
                break
            except GithubException as e:
                print(e.status)
                print('General exception --', slug)
                break
            except r.exceptions.ConnectionError as e:
                print('Retries limit exceeded --', slug)
                print(str(e))
                time.sleep(10)
                continue
            except r.exceptions.Timeout as e:
                print('Time out exception --', slug)
                print(str(e))
                time.sleep(10)
                continue
            break
        df_repo_stats.to_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_stats_practice1.csv', index = False)        

In [24]:
pull_repo_stats()

In [26]:
df_repo_stats = pd.read_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_stats_practice1.csv')
print(df_repo_stats.head())
print(df_repo_stats["stars"])

                                      slug  stars
0                  dbuskariol-org/chromium    0.0
1                                 cjbd/src    NaN
2  firemax13/android_kernel_sm6150_unified    NaN
3                      firemax13/a80kernel    NaN
4  eugene-matvejev/ultimate-commit-machine    1.0
0      0.0
1      NaN
2      NaN
3      NaN
4      1.0
5    119.0
6      0.0
7      2.0
8      0.0
9      1.0
Name: stars, dtype: float64


In [304]:
len(star)

8776

In [305]:
raw_slug_data["stars"] = star

In [306]:
print(raw_slug_data.head())

                                 id        spdx  \
0  MDEwOlJlcG9zaXRvcnkxODA1NjY0NzE=  Apache-2.0   
1  MDEwOlJlcG9zaXRvcnkxMzQyMzY5NTk=  Apache-2.0   
2  MDEwOlJlcG9zaXRvcnk1OTc5OTQxOQ==  Apache-2.0   
3  MDEwOlJlcG9zaXRvcnkzMDkxNTc1MDI=  Apache-2.0   
4  MDEwOlJlcG9zaXRvcnkxOTYwNTU3Nw==  Apache-2.0   

                            slug           createdat  \
0  kderme/spark-memory-simulator 2019-04-10 11:22:43   
1     zhangyineng/pentaho_kettle 2018-05-21 07:55:12   
2          zorba-processor/zorba 2016-05-27 02:56:03   
3                  jsoniq/jsoniq 2020-11-01 18:08:05   
4                   28msec/zorba 2014-05-09 09:13:04   

                       description primarylanguage  \
0                             None           Scala   
1                             None            Java   
2      Zorba - the NoSQL processor             C++   
3  JSONiq: The JSON Query Language             C++   
4  JSONiq & XQuery Query Processor             C++   

                               

In [307]:
raw_slug_data.to_csv(r'/home/zz3hs/git/dspg21oss/output/repo_slugs_stars_commits_bt_10000_20000.csv', index = False)

In [319]:
df = pd.read_csv('/home/zz3hs/git/dspg21oss/output/repo_slugs_stars_commits_bt_10000_20000.csv')

In [320]:
#print(df.columns)
#print(df.describe())
#print(df.shape)
#df.info
df[["slug", "stars"]]

,slug,stars
0,kderme/spark-memory-simulator,0
1,zhangyineng/pentaho_kettle,0
2,zorba-processor/zorba,29
3,jsoniq/jsoniq,0
4,28msec/zorba,113
...,...,...
8771,xiaoxueyong/my-bootstrap,0
8772,khoahuong/Bootstrap,0
8773,EvanD7657/WIP-Clicking-Game,0
8774,inspades/bootstrap,0


In [323]:
#repo_slug_R_stars_100
#https://github.com/jimtheflash/breftools/
df_star_na = df[df["stars"].isna()]
print(df_star_na.shape) 
print(7397/8776) #percent repo with na stars

0.8428669097538742

In [350]:
#save repos with valid star numbers 
df_star = df.dropna(subset=["stars"])
print(df_star.shape) 

(1379, 11)


In [1]:
#repo_slug_R_stars
#tb = pd.crosstab(index =df['stars'], columns = 'count')
#tb.sort_values(by=['stars'], ascending=False)


#3932/5000 #na

#na urls
df_star_na.head()
#na but page exists
#print(df.iloc[4995]["slug"])
#https://github.com/egeor90/stocks/
#print(df[df["slug"] ==  "egeor90/stocks"])

#pages fail to identify
#https://github.com/XSanC/FLink
#https://github.com/JakaMohorko/MastersDissFlink

#actual not found pages
#https://github.com/tdudzik/dotty-test

NameError: name 'df_star_na' is not defined

In [328]:
slugs = df_star_na["slug"]
print(slugs)

18                 tdudzik/dotty-test
76                        XSanC/FLink
77                     devyaniB/Flink
78              craiig/spark-keychain
79       JakaMohorko/MastersDissFlink
                    ...              
8692           UnicoinUWA/go-ethereum
8693    LegendaryCoinTech/go-ethereum
8694                     FFTEAM/oscam
8695                 peter-gyq/mpc-hc
8696                      EricIO/helm
Name: slug, Length: 7397, dtype: object


In [329]:
star = []
for slug in slugs:
    url =f'https://github.com/{slug}/'
    star.append(stars_count(url))
    
    
#https://github.com/AlbertoMiliano/Statistics/
#https://github.com/codez0mb1e/HomeCreditDefaultRisk
print(star)
#2 na's out of 7 previoursly na's

[None, '0', '0', '2', '0', '0', '0', '0', '0', '0', '84', '13.6k', '15', '0', '0', '0', '0', '7', '20', '0', '1', '1', '0', '1', '0', '8', '4.6k', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '10', '0', '0', '1', '343', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '28.2k', '0', '0', '0', '0', '0', '0', '0', '113', '3', '0', '0', '0', '400', '0', '0', '0', '31', '0', '10', '0', '0', '161', '13', '3', '0', '0', '0', '0', '23', '0', '0', '0', '0', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, N

In [337]:
df_star_na = df_star_na.drop(columns = "stars")
df_star_na["stars"] = star

In [351]:
#save repos with valid star numbers 
df_star_2 = df_star_na.dropna(subset=["stars"])
print(df_star_2.shape) 

(1301, 11)


In [340]:
df_star_na_2 = df_star_na[df_star_na["stars"].isna()]
print(df_star_na_2.shape) 
print(6096/7397) #82 percent repos have na's on the second round of tries comparing to the first round
print(6096/8776) #69 percent repos have na's on the second round of tries comparing to the original

(6096, 11)
0.8241178856293092
0.6946216955332726


In [341]:
df_star_na_2.head()

,id,spdx,slug,createdat,description,primarylanguage,branch,commits,asof,status,stars
18,MDEwOlJlcG9zaXRvcnkzMjUwNDcwOTI=,Apache-2.0,tdudzik/dotty-test,2020-12-28 15:26:23,NaN,Scala,MDM6UmVmMzI1MDQ3MDkyOnJlZnMvaGVhZHMvbWFpbg==,19971,2021-01-03 18:33:16,Init,None
185,MDEwOlJlcG9zaXRvcnkxMDQ3NDkwMDU=,GPL-3.0,ahmad18189/erp_test2,2017-09-25 12:47:41,test,Python,MDM6UmVmMTA0NzQ5MDA1OnJlZnMvaGVhZHMvZWRpdHM=,19603,2021-01-03 17:59:15,Init,None
186,MDEwOlJlcG9zaXRvcnkxNjkxNzU3NDg=,GPL-3.0,bhavikpatel7023/ERPNext_v8,2019-02-05 01:27:36,NaN,Python,MDM6UmVmMTY5MTc1NzQ4OnJlZnMvaGVhZHMvbWFzdGVy,19598,2021-01-03 14:20:51,Init,None
187,MDEwOlJlcG9zaXRvcnk3MjA2NDE4OQ==,Apache-2.0,shubhramittal/selenium,2016-10-27 02:40:36,NaN,JavaScript,MDM6UmVmNzIwNjQxODk6cmVmcy9oZWFkcy9tYXN0ZXI=,19597,2021-01-03 15:22:53,Init,None
188,MDEwOlJlcG9zaXRvcnkzMDE0NjE5NDA=,Apache-2.0,TeamChameleonDev/glitch-community,2020-10-05 15:52:48,NaN,JavaScript,MDM6UmVmMzAxNDYxOTQwOnJlZnMvaGVhZHMvbWFzdGVy,19595,2021-01-03 22:25:59,Init,None


In [352]:
#3rd try
slugs = df_star_na_2["slug"]
star = []
for slug in slugs:
    url =f'https://github.com/{slug}/'
    star.append(stars_count(url))
print(star)

[None, '0', '0', '0', '0', '79', '0', '0', '1', '0', '0', '1', '4k', '1', '646', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '308', '0', '0', '0', '3', '0', '8', '0', '331', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '1', '21.9k', '0', '0', '0', '2', '0', '0', '0', '0', '0', '1', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, N

In [353]:
#fill in na's with newly scrapped data
df_star_na_2 = df_star_na_2.drop(columns = "stars")
df_star_na_2["stars"] = star


In [354]:
#save repos with valid star numbers 
df_star_3 = df_star_na_2.dropna(subset=["stars"])
print(df_star_3.shape) 

(1013, 11)


In [ ]:

#df_star_na_2 = df_star_na[df_star_na["stars"].isna()]
#print(df_star_na_2.shape) 
#print(6096/7397) #82 percent repos have na's on the second round of tries comparing to the first round
#print(6096/8776) #69 percent repos have na's on the second round of tries comparing to the original


In [357]:
frames = [df_star, df_star_2, df_star_na_2]
df_final = pd.concat(frames)
df_final.shape

(8776, 11)

In [358]:
df_final.to_csv(r'/home/zz3hs/git/dspg21oss/output/repo_slugs_stars_commits_bt_10000_20000_clean.csv', index = False)

In [360]:
print(df_final.head())
#save repos with valid star numbers 
check = df_final.dropna(subset=["stars"])
print(check.shape) 

                                 id        spdx  \
0  MDEwOlJlcG9zaXRvcnkxODA1NjY0NzE=  Apache-2.0   
1  MDEwOlJlcG9zaXRvcnkxMzQyMzY5NTk=  Apache-2.0   
2  MDEwOlJlcG9zaXRvcnk1OTc5OTQxOQ==  Apache-2.0   
3  MDEwOlJlcG9zaXRvcnkzMDkxNTc1MDI=  Apache-2.0   
4  MDEwOlJlcG9zaXRvcnkxOTYwNTU3Nw==  Apache-2.0   

                            slug            createdat  \
0  kderme/spark-memory-simulator  2019-04-10 11:22:43   
1     zhangyineng/pentaho_kettle  2018-05-21 07:55:12   
2          zorba-processor/zorba  2016-05-27 02:56:03   
3                  jsoniq/jsoniq  2020-11-01 18:08:05   
4                   28msec/zorba  2014-05-09 09:13:04   

                       description primarylanguage  \
0                              NaN           Scala   
1                              NaN            Java   
2      Zorba - the NoSQL processor             C++   
3  JSONiq: The JSON Query Language             C++   
4  JSONiq & XQuery Query Processor             C++   

                         